In [3]:
import pandas as pd
import numpy as np 

import torch 
import torchvision.models as models
from torchvision import datasets, transforms
from torch import nn
from torch import optim

from PIL import Image
from tqdm import tqdm

import matplotlib.pyplot as plt

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [24]:
def accuracy(model_output, test_target):
    """Return the accuracy of the model output."""
    nb_samples = model_output.shape[0]
    output_int = torch.zeros(nb_samples)
    
    # Convert probability to decision
    output_int = torch.argmax(model_output, 1)
    
    nb_errors = (output_int - test_target).type(torch.BoolTensor).sum().item()
    return (nb_samples - nb_errors) / nb_samples

In [36]:
class Webnet(nn.Module):
    def __init__(self):
        super(Webnet, self).__init__()
        
        resnet = models.resnet18(pretrained=True)
        
        self.features = torch.nn.Sequential(*list(resnet.children())[:-1])
        self.fc = torch.nn.Linear(512,16)

    def forward(self, x):
        x = self.features(x).reshape(-1, 512)
        x = self.fc(x)
        return x

In [37]:
batch_size = 100

crop_perc = 0.1
in_dim = np.array([960, 540])

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
crop = transforms.RandomCrop(size=in_dim * crop_perc)
transform = transforms.Compose([transforms.ToTensor(), crop, normalize])
tensorize = transforms.ToTensor()

data_transforms = {
    'train': transforms.Compose([crop, tensorize, normalize]),
    'valid': transforms.Compose([crop, tensorize, normalize])
}

path = '/dlabdata1/lugeon/websites_alexa_mp2_468_16cat_screenshots/'

data_images = {x: datasets.ImageFolder(os.path.join(path, x), data_transforms[x]) for x in ['train', 'valid']}

dataloaders_dict = {x: torch.utils.data.DataLoader(data_images[x], 
                                                   batch_size=batch_size, 
                                                   shuffle=True, 
                                                   num_workers=4) for x in ['train', 'valid']}


In [38]:
try:
    model = Webnet().to(device)

    nb_epochs = 50

    #torch.set_num_threads(24)

    criterion = nn.CrossEntropyLoss()

    optimizer = optim.Adam(model.parameters(), 1e-3)

    for e in range(nb_epochs):
        
        print('Epoch {}'.format(e))
        
        for p in model.features.parameters():
            p.requires_grad = False
            
        for phase in ['train', 'valid']:

            if phase == 'train':
                model.train() 
            else:
                model.eval() 
                
            running_loss = 0.0
            running_corrects = 0

            for data in tqdm(iter(dataloader[phase])):

                inputs = data[0].to(device)
                targets = data[1].to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):

                    outputs = model(inputs)
                    loss = criterion(outputs, targets)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(outputs == targets.data)
                    
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
                        
        print('  {} loss: {:.3f}    accuracy: {:.3f}'.format(phase, epoch_loss, epoch_acc))
        
finally:
    torch.cuda.empty_cache()

100%|██████████| 45/45 [00:45<00:00,  1.02s/it]

Epoch 0 --- Loss: 2.776     Accuracy: 0.090



100%|██████████| 45/45 [00:47<00:00,  1.06s/it]

Epoch 1 --- Loss: 2.683     Accuracy: 0.127



100%|██████████| 45/45 [00:49<00:00,  1.10s/it]

...

Epoch 47 --- Loss: 2.182     Accuracy: 0.313



100%|██████████| 45/45 [00:48<00:00,  1.07s/it]

Epoch 48 --- Loss: 2.168     Accuracy: 0.325



100%|██████████| 45/45 [00:47<00:00,  1.06s/it]

Epoch 49 --- Loss: 2.168     Accuracy: 0.328
